In [ ]:
import numpy as np
from fastapi import FastAPI, File, UploadFile
import uvicorn
from io import BytesIO
from PIL import Image
import tensorflow as tf

# create object
app = FastAPI()

MODEL = tf.keras.models.load_model("../saved_models/cataract_model")
CLASS_NAMES = ["Cataract", "Normal"]

@app.get("/ping")
async def ping():
    return "Hello I am Nuri"

def read_file_as_image(data) -> np.ndarray:
    image = np.array(Image.open(BytesIO(data)))
    return image

@app.post("/prediksi")
async def predict(
        upload: UploadFile = File(...)
):
    image = read_file_as_image(await upload.read())
    img_batch = np.expand_dims(image, 0)

    predictions = MODEL.predict(img_batch)

    predicted_class = CLASS_NAMES[np.argmax(predictions[0])]
    np.max(predictions[0])
    confidence = np.max(predictions[0])
    return {
        'class' : predicted_class,
        'confidence' : float(confidence)
    }

if __name__ == "__main__":
    uvicorn.run(app, host='localhost', port=8000)